In [1]:
import numpy as np
import sympy as sp

In [2]:
# function to calculate number of occurrence of a pattern in a sub-config including overlapping ones and circular occurrences
def countOccurrencesCircular(string, substring):
    cnt = 0
    x = string + string
    m = len(substring)
    for i in range(len(string)):
        if substring == x[i: i + m]:
            cnt += 1
    return cnt


In [3]:
# function to calculate number of occurrence of a pattern in a sub-config including overlapping ones
def countOccurrences(sub, string):
    count = start = 0
    while True:
        start = string.find(sub, start) + 1
        if start > 0:
            count += 1
        else:
            return count


In [4]:
# test
print(countOccurrences("10", "010101111"))

2


In [5]:
# function to calculate probability of conversion of sub-config A to B
def calcProp(A, B):
    a = len(A)
    b = len(B)
    if (a != b or A[0] != B[0] or A[a - 1] != B[b - 1]):
        return 0
    r, p = sp.symbols("r p")
    r = 1
    for i in range(1, a - 1):    # ignoring first and last cell
        if (A[i] == B[i]):    # must remain on its strategy
            if (A[i - 1] == A[i] or A[i] == A[i + 1]):    # checking if the ith cell can flip
                r *= (1 - p)
            else:
                r *= 1
        else:
            if (A[i - 1] == A[i] or A[i] == A[i + 1]):
                r *= p
            else:
                return 0
    return r


In [6]:
# function to decide if a cell is able to switch its strategy regarding transition rules
def decideIfTheCellCanFlip(left, center, right, transition_rules):
    n = transition_rules.shape[0]
    for i in range(n):
        if (transition_rules[i][0][0] == left and transition_rules[i][0][1] == center and transition_rules[i][0][2] == right and transition_rules[i][1][0] != center):
            return True
    return False

In [8]:
# general function to calculate probability of conversion of sub-config A to B regarding certain transition rules
def generalCalcProb(A, B, transition_rules = np.array([("000", "1"), ("001", "1"), ("011", "0"), ("100", "1"), ("110", "0"), ("111", "0")])):
    a = len(A)
    b = len(B)
    if (a != b or A[0] != B[0] or A[a - 1] != B[b - 1]):
        return 0
    r, p = sp.symbols("r p")
    r = 1
    for i in range(1, a - 1):    # ignoring first and last cell
        if (A[i] == B[i]):    # must remain on its strategy
            if (decideIfTheCellCanFlip(A[i - 1], A[i], A[i + 1], transition_rules)):    # checking if the ith cell can flip
                r *= (1 - p)
            else:
                r *= 1
        else:
            if (decideIfTheCellCanFlip(A[i - 1], A[i], A[i + 1], transition_rules)):
                r *= p
            else:
                return 0
    return r


In [9]:
# test
print(calcProp("011110", "010110"))
print(generalCalcProb("011110", "010110"))

p*(1 - p)**3
p*(1 - p)**3


In [10]:
def generateAllBinaryStrings(length):
    arr = [""] * 2 ** length
    form = '#0' + str(length + 2) + 'b'
    for i in range(2 ** length):
        arr[i] = format(i, form)[2:]
    return np.array(arr)


In [11]:
# test
print(generateAllBinaryStrings(3))

['000' '001' '010' '011' '100' '101' '110' '111']


In [12]:
# function to calculate the expected difference of number of a list of patterns in a sub-config
def calcExpectedDifference(patterns, sub_config, transition_rules = np.array([("000", "1"), ("001", "1"), ("011", "0"), ("100", "1"), ("110", "0"), ("111", "0")])):
    k = len(sub_config) - 2
    n = 2 ** k
    binary_strings = generateAllBinaryStrings(k)
    number_of_patterns = patterns.shape[0]
    expected_differences = [0] * number_of_patterns
    for j in range(number_of_patterns):
        #
        total_expectation = 0
        current_number_of_occurrence = countOccurrences(patterns[j], sub_config[1:-1])
        for i in range(n):
            total_expectation += generalCalcProb(sub_config, sub_config[0] + binary_strings[i] + sub_config[-1], transition_rules) * (countOccurrences(patterns[j], binary_strings[i]) - current_number_of_occurrence)
        #
        expected_differences[j] = total_expectation
    return np.array(expected_differences)


In [13]:
# test
calcExpectedDifference(np.array(["001"]), "0010010")[0]

-p**3 - 3*p**2*(1 - p) - 2*p*(1 - p)**2

In [14]:
#
def getAllDifferencesOnExpectationWithGivenPatterns(patterns, transition_rules = np.array([("000", "1"), ("001", "1"), ("011", "0"), ("100", "1"), ("110", "0"), ("111", "0")])):
    pattern_max_length = 0
    m = patterns.shape[0]
    #check if all patterns have same length
    for i in range(m):
        if (len(patterns[i]) > pattern_max_length):
            pattern_max_length = len(patterns[i])

    # binary sub-config length should be equal with 2 * pattern_length + 1
    k = 2 * pattern_max_length + 1
    n = 2 ** k
    binary_sub_configs = generateAllBinaryStrings(k)
    diffs = [ [0] * m for i in range(n)]
    for j in range(n):
        expectations_of_current_sub_config = calcExpectedDifference(patterns, binary_sub_configs[j], transition_rules)
        diffs[j] = expectations_of_current_sub_config
    
    return np.array(diffs).T


In [20]:
pats = np.array(["0", "010", "01011"])
tr = np.array([("001", "1"), ("010", "0")])
x = getAllDifferencesOnExpectationWithGivenPatterns(pats, tr)
y = np.dot(np.array([3, 3, 1]), x)



In [23]:
for i in y:
    print(i)

0
-3*p
3*p**2
-3*p
-6*p*(1 - p)
-6*p*(1 - p)
-3*p
-3*p
-6*p*(1 - p)
-3*p**3 - 12*p**2*(1 - p) - 9*p*(1 - p)**2
3*p**3 - 3*p*(1 - p)**2
-6*p*(1 - p)
-3*p
-3*p
-3*p
-3*p
-6*p*(1 - p)
-3*p**3 - 12*p**2*(1 - p) - 9*p*(1 - p)**2
3*p**4 - 9*p**2*(1 - p)**2 - 5*p*(1 - p)**3
-3*p**3 - 12*p**2*(1 - p) - 8*p*(1 - p)**2
-6*p**2*(1 - p) - 6*p*(1 - p)**2
-6*p**2*(1 - p) - 6*p*(1 - p)**2
-p**2 - 8*p*(1 - p)
-p**2 - 8*p*(1 - p)
-3*p
-6*p**2 - 6*p*(1 - p)
0
-3*p
-3*p
-3*p
-3*p
-3*p
-6*p*(1 - p)
-3*p**3 - 12*p**2*(1 - p) - 9*p*(1 - p)**2
3*p**4 - 9*p**2*(1 - p)**2 - 6*p*(1 - p)**3
-3*p**3 - 12*p**2*(1 - p) - 9*p*(1 - p)**2
-12*p**3*(1 - p) - 24*p**2*(1 - p)**2 - 11*p*(1 - p)**3
-12*p**3*(1 - p) - 24*p**2*(1 - p)**2 - 11*p*(1 - p)**3
-3*p**3 - 12*p**2*(1 - p) - 8*p*(1 - p)**2
-3*p**3 - 12*p**2*(1 - p) - 8*p*(1 - p)**2
-6*p**2*(1 - p) - 6*p*(1 - p)**2
-3*p**4 - 15*p**3*(1 - p) - 21*p**2*(1 - p)**2 - 9*p*(1 - p)**3
3*p**4 + 3*p**3*(1 - p) - 3*p**2*(1 - p)**2 - 3*p*(1 - p)**3
-6*p**2*(1 - p) - 6*p*(1 - p)*

In [17]:
# test
5 * calcExpectedDifference(np.array(["1"]), "0000010", tr)[0]

0

In [15]:
def getIntervalInWhichAllAreNegative(expectations):
    p = sp.symbols('p')
    lower_bound = sp.solvers.inequalities.solve_univariate_inequality(p > 0, p)
    upper_bound = sp.solvers.inequalities.solve_univariate_inequality(p < 1, p)
    interval = sp.And(lower_bound, upper_bound)
    interval = sp.simplify(interval)
    for i in range(expectations.shape[0]):
        if expectations[i] != 0:
            beta = sp.solvers.inequalities.solve_univariate_inequality(expectations[i] < 0, p)
            interval = sp.And(interval, beta)
            interval = sp.simplify(interval)
            if interval == sp.false:
                return sp.false
    return interval

In [ ]:
pats = np.array(["000", "001", "010", "011", "100", "101", "110", "111"])
x = getAllDifferencesOnExpectationWithGivenPatterns(pats)
t1 = 1
t2 = 11
t3 = 5
for i1 in range(t1, t2, t3):
    for i2 in range(t1, t2, t3):
        for i3 in range(-t2, -t1, t3):
            for i4 in range(t1, t2, t3):
                for i5 in range(t1, t2, t3):
                    for i6 in range(-t2, -t1, t3):
                        for i7 in range(t1, t2, t3):
                            for i8 in range(t1, t2, t3):
                                y = np.dot(np.array([i1, i2, i3, i4, i5, i6, i7, i8]), x)
                                print(str(i1) + ", " + str(i2) + ", " + str(i3) + ", " + str(i4) + ", " + str(i5) + ", " + str(i6) + ", " + str(i7) + ", " + str(i8) + ": " + str(getIntervalInWhichAllAreNegative(y)))